## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk
import string

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords

from nltk import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sms_spam = '/content/drive/MyDrive/sms_spam.csv'

In [ ]:
data = pd.read_csv(sms_spam)

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def text_data_preprocessing(x):
    tknzr = TweetTokenizer()
    tokenized_text = pd.Series(tknzr.tokenize(i.lower()) for i in x)

    noise = stopwords.words('english')
    t_t_without_noise = pd.Series([x for x in i if x not in noise] for i in tokenized_text)

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = pd.Series([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in i] for i in t_t_without_noise)

    processed_text = pd.Series(" ".join(i) for i in lemmatized_text)

    for i in range(len(processed_text)):
        for ch in string.punctuation:
            processed_text[i] = processed_text[i].replace(ch, "")

    return processed_text

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.text, data.type, train_size=0.7)

x_train_processed = text_data_preprocessing(x_train)

x_test_processed = text_data_preprocessing(x_test)

In [ ]:
# CountVectorizer. Words

pipeline = Pipeline([
           ('vect', CountVectorizer()),
           ('clf', MultinomialNB()),
])

params = {
    'vect__ngram_range': [(k, i) for i in range(2, 6) for k in range(1, i + 1)]
}

search = GridSearchCV(pipeline, params, scoring='f1_macro', n_jobs=-1)
search.fit(x_train_processed, y_train)
print(search.best_score_)
print(search.best_params_)

pred = search.predict(x_test_processed)
print(classification_report(y_test, pred))

0.9635609102873941
{'vect__ngram_range': (1, 3)}
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1460
        spam       0.98      0.90      0.94       208

    accuracy                           0.99      1668
   macro avg       0.98      0.95      0.97      1668
weighted avg       0.99      0.99      0.99      1668



In [ ]:
pipeline2 = Pipeline([
           ('vect', CountVectorizer(analyzer='char')),
           ('clf', MultinomialNB()),
])

params2 = {
    'vect__ngram_range': [(k, i) for i in range(2, 8) for k in range(1, i + 1)]
}

search2 = GridSearchCV(pipeline2, params2, scoring='f1_macro',  n_jobs=-1)
search2.fit(x_train_processed, y_train)
print(search2.best_score_)
print(search2.best_params_)

pred2 = search2.predict(x_test_processed)
print(classification_report(y_test, pred2))

0.9652777377539685
{'vect__ngram_range': (3, 3)}
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1460
        spam       0.92      0.93      0.93       208

    accuracy                           0.98      1668
   macro avg       0.96      0.96      0.96      1668
weighted avg       0.98      0.98      0.98      1668



In [ ]:
# TfidfVectorizer

pipeline3 = Pipeline([
           ('vect', TfidfVectorizer()),
           ('clf', MultinomialNB()),
])

params3 = {
    'vect__ngram_range': [(k, i) for i in range(2, 6) for k in range(1, i + 1)],
    'vect__min_df': [4, 5, 2],
    'vect__max_df': [0.75, 0.85, 0.65],
    'vect__max_features': [4000, 5000, 6000]
}


search3 = GridSearchCV(pipeline3, params3, scoring='f1_macro', n_jobs=-1)
search3.fit(x_train_processed, y_train)
print(search3.best_score_)
print(search3.best_params_)

0.939813126329159
{'vect__max_df': 0.75, 'vect__max_features': 4000, 'vect__min_df': 4, 'vect__ngram_range': (1, 2)}


In [ ]:
pred3 = search3.predict(x_test_processed)
print(classification_report(y_test, pred3))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1460
        spam       0.99      0.83      0.91       208

    accuracy                           0.98      1668
   macro avg       0.99      0.92      0.95      1668
weighted avg       0.98      0.98      0.98      1668



In [ ]:
report = classification_report(y_test, pred).split()
report2 = classification_report(y_test, pred2).split()
report3 = classification_report(y_test, pred3).split()

d = {'vectorizer': ['CountVectorizer(1,3) words', 'CountVectorizer(3,3) chars', 'TfidfVectorizer'],
     'precision': [report[19], report2[19], report3[19]],
     'recall': [report[20], report2[20], report3[20]],
     'f1-score': [report[21], report2[21], report3[21]],
     'accuracy': [report[15], report2[15], report3[15]]}
df = pd.DataFrame(d)
df

,vectorizer,precision,recall,f1-score,accuracy
0,"CountVectorizer(1,3) words",0.98,0.95,0.97,0.99
1,"CountVectorizer(3,3) chars",0.96,0.96,0.96,0.98
2,TfidfVectorizer,0.99,0.92,0.95,0.98


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb') 
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
res = re.findall(r'\b[a-zA-Z]{2}', "Fuck you asshole, im done with this shit!")
res

['Fu', 'yo', 'as', 'im', 'do', 'wi', 'th', 'sh']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
res2 = re.split(r'\.', 'I want to break free. How to draw a cat. Nobody knows.I want to finish this ex. Please.', maxsplit=2) 
print(res2)

['I want to break free', ' How to draw a cat', ' Nobody knows.I want to finish this ex. Please.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
res3 = re.sub(r'\d','DIG', 'I want to see your 11 monkeys or 2 dogs or 343 lions')
res3

'I want to see your DIGDIG monkeys or DIG dogs or DIGDIGDIG lions'

In [ ]:
res3 = re.sub(r'\ [0-9]{1}\ ',' DIG ', 'I want to see your 11 monkeys or 2 dogs or 343 lions')
res3

'I want to see your 11 monkeys or DIG dogs or 343 lions'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
res4 = re.sub(r'http[s]{,1}:\/\/\S+','', 'U can find my labs on https://drive.google.com/drive/u/0/my-drive')
res4

'U can find my labs on '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[А-Яа-яё\-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog2 = re.compile(r'@[^\^\$\*\+\?\{\}\[\]\\\|\(\)\@\ ]{2,}\.\w{2,4}\b')
prog2.findall("abc.test@gmail.comjhkjk, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.bizghjk")

['@test.in', '@analyticsvidhya.com']